In [1]:
## from stefans
# import packages
import gc
import glob
import cmcrameri.cm as ccm
import matplotlib as mpl
from matplotlib import colors
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import os
import matplotlib
import pylab as P

from params import savepath, homedir, allnames, datadir
import cartopy.crs as crs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
plt.style.use('science.mplstyle')

states = gpd.read_file('../data/ne_110m_admin_1_states_provinces.shp')

In [2]:

gcms_dict = {'d01': ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
                     'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
                     'canesm5','access-cm2','ec-earth3'],
             'd02': ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
                     'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
                     'canesm5','access-cm2','ec-earth3'],
             'd03': ['ec-earth3-veg'],
             'd04': ['ec-earth3-veg'],}

variants_dict = {'d01': ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
                         'r1i1p1f1','r1i1p1f1','r2i1p1f2',
                         'r1i1p2f1','r5i1p1f1','r1i1p1f1',],
                 'd02': ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
                         'r1i1p1f1','r1i1p1f1','r2i1p1f2',
                         'r1i1p2f1','r5i1p1f1','r1i1p1f1',],
                 'd03': ['r1i1p1f1'],
                 'd04': ['r1i1p1f1'],}

calendars_dict = {'d01': ['365_day','proleptic_gregorian','proleptic_gregorian',
                         'proleptic_gregorian','365_day','360_day',
                         '365_day','proleptic_gregorian','proleptic_gregorian',],
                 'd02': ['365_day','proleptic_gregorian','proleptic_gregorian',
                         'proleptic_gregorian','365_day','360_day',
                         '365_day','proleptic_gregorian','proleptic_gregorian',],
                 'd03': ['proleptic_gregorian'],
                 'd04': ['proleptic_gregorian'],}

ssps_dict = {'d01': ['ssp370','ssp370','ssp370','ssp370',
                     'ssp370','ssp370','ssp370','ssp370',
                     'ssp370',],
             'd02': ['ssp370','ssp370','ssp370','ssp370',
                     'ssp370','ssp370','ssp370','ssp370',
                     'ssp370',],
             'd03': ['ssp370'],
             'd04': ['ssp370'],}


source_ids = {'access-cm2':'ACCESS-CM2', 
             'cesm2':'CESM2',
             'cnrm-esm2-1':'CNRM-ESM2-1', 
             'canesm5':'CanESM5',
             'ec-earth3':'EC-Earth3',
             'ec-earth3-veg': 'EC-Earth3-Veg',
             'fgoals-g3':'FGOALS-g3', 
             'mpi-esm1-2-lr':'MPI-ESM1-2-LR',
             'ukesm1-0-ll':'UKESM1-0-LL'}

date_start_hist, date_end_hist = "1980-09-01", "2014-08-31"
date_start_ssp, date_end_ssp = "2014-09-01", "2100-08-31"
dir_meta = '/glade/campaign/uwyo/wyom0112/postprocess/meta/meta_new/'


In [3]:
tas_ts = pd.read_csv('wrfrun_globalavg_temp.csv', index_col = 0, parse_dates = True)
mask = (tas_ts['year'] >= 1980) & (tas_ts['year'] <= 2000)
baseline = tas_ts[mask].groupby('year').mean(numeric_only = True).mean().values[0]
tas_ts['d_tas'] = tas_ts['tas'] - baseline
tas_ts = tas_ts.groupby(['year', 'source_id']).mean()
tas_ds = xr.Dataset.from_dataframe(tas_ts[['d_tas']])

/glade/scratch/mcowherd/ipykernel_295624/2899011245.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tas_ts = tas_ts.groupby(['year', 'source_id']).mean()
